$$A x = b, \quad A = \begin{pmatrix} a_{11} & a_{12} & \cdots & a_{1n} \\ a_{21} & a_{22} & \cdots & a_{2n} \\ \vdots & \vdots & \ddots & \vdots \\ a_{n1} & a_{n2} & \cdots & a_{nn} \end{pmatrix}, \quad b = \begin{pmatrix} b_1 \\ b_2 \\ \vdots \\ b_n \end{pmatrix}$$

A为非奇异矩阵，求向量x

Gauss消去法思路：

系数矩阵逐步消元得到上三角矩阵，然后回代求解。

$$
\left\{\begin{array}{l}
a_{11}^{(1)} x_1+a_{12}^{(1)} x_2+\ldots+a_{1 n}^{(1)} x_n=b_1^{(1)} \\
a_{21}^{(1)} x_1+a_{22}^{(1)} x_2+\ldots+a_{2 n}^{(1)} x_n=b_2^{(1)} \\
\ldots \\
a_{n 1}^{(1)} x_1+a_{n 2}^{(1)} x_2+\ldots+a_{n n}^{(1)} x_n=b_n^{(1)}
\end{array}\right.
$$


令 $A^{(1)} x=b^{(1)}$ ，其中 $A^{(1)}=A, \quad b^{(1)}=b$

若 $a_{11}^{(1)} \neq 0$ ，令 $l_{i 1}=\frac{a_{i 1}^{(1)}}{a_{11}^{(1)}}$ ，
再以第 1 行乘 $-l_{i 1}$ 加到第 $i$ 行 $(i=2,3, \cdots, n)$ 上。 $A^{(1)} \rightarrow A^{(2)}, b^{(1)} \rightarrow b^{(2)}$

$$
\begin{gathered}
a_{i j}^{(2)}=a_{i j}^{(1)}-l_{i 1} a_{1 j}^{(1)}(i, j=2,3, \cdots, n) \\
b_i^{(2)}=b_i^{(1)}-l_{i 1} b_1^{(1)}(i=2,3, \cdots, n)
\end{gathered}
$$


若 $a_{k k}^{(k)} \neq 0$ ，令 $l_{i k}=\frac{a_{i k}^{(k)}}{a_{k k}^{(k)}}$ ，
第 $k$ 行乘 $-l_{i k}$ 加到第 $i$ 行 $(i=k+1, k+2, \cdots, n)$ 上，

$$
\begin{gathered}
a_{i j}^{(k+1)}=a_{i j}^{(k)}-l_{i k} a_{k j}^{(k)}(i, j=k+1, k+2, \cdots, n) \\
b_i^{(k+1)}=b_i^{(k)}-l_{i k} b_k^{(k)}(i=k+1, k+2, \cdots, n)
\end{gathered}
$$


In [3]:
import numpy as np


def reg_utm(A, b):
    """ 解上三角方程组 回代算法 """
    if np.any(np.diag(A) == 0):
        raise ValueError("Error: 对角线上有0元素，不能使用该函数")

    n = len(b)
    X = np.zeros(n)
    X[n - 1] = b[n - 1] / A[n - 1, n - 1]

    for k in range(n - 2, -1, -1):
        t = np.dot(A[k, k + 1:n], X[k + 1:n])
        X[k] = (b[k] - t) / A[k, k]

    return X

In [4]:
def gsem_base(A, b):
    """
    Gauss消去法
    A : 系数矩阵
    b : 右端常数 列向量
    X : 求得的解向量
    Ae: 得到的上三角矩阵
    be: 对应的右端项
    """
    A1 = np.hstack((A, b.reshape(-1, 1)))  # 增广矩阵
    n = len(A)
    n1 = A1.shape[1]

    # 系数矩阵化为上三角
    for i in range(n - 1):
        if A1[i, i] != 0:
            for k in range(i + 1, n):
                lik = A1[k, i] / A1[i, i]  # 算子
                A1[k, i:n1] -= lik * A1[i, i:n1]
        else:
            raise ValueError("出现零元素，不能使用该函数")

    # 得到上三角后进行回代
    Ae = A1[:, :n]
    be = A1[:, n1 - 1]
    X = reg_utm(Ae, be)

    return X, Ae, be


# 测试具体矩阵
A = np.array([[3, 2, -4],
              [2, 3, 3],
              [5, -3, 1]], dtype=float)

b = np.array([3, 15, 14], dtype=float)

# 使用 gsem_base 函数求解
X, Ae, be = gsem_base(A, b)

# 输出结果
print("解向量 X:", X)
print("上三角矩阵 Ae:\n", Ae)
print("对应的右端项 be:", be)

# 验证结果
result = np.dot(A, X)
print("验证：A @ X =", result)
print("应等于 b:", b)

解向量 X: [3. 1. 2.]
上三角矩阵 Ae:
 [[ 3.          2.         -4.        ]
 [ 0.          1.66666667  5.66666667]
 [ 0.          0.         29.2       ]]
对应的右端项 be: [ 3.  13.  58.4]
验证：A @ X = [ 3. 15. 14.]
应等于 b: [ 3. 15. 14.]


然而这种高斯消去的步骤存在缺点。
比如当 $a_{kk}^{(k)}$为0时不能使用。

且 $\left|a_{kk}^{(k)}\right|\approx0$ 或相对其他元素较小时，舍入误差增加，可能导致误差剧增。

列主元消去：每一步消元前，先找到列中最大元素所在的那一行，并与原来的行交换位置


全主元消去：每一步消元前，先找到剩余矩阵中最大的元素，做对应的行交换与列交换（注意由于进行了列交换，最后要把$X$的位置对应回去）。

In [5]:
def gsem_column(A, b):
    """
    列主元消去法
    A : 系数矩阵
    b : 右端常数 [列向量]
    X : 求得的解向量
    Ae: 得到的上三角矩阵
    be: 对应的右端项
    """
    A1 = np.hstack((A, b.reshape(-1, 1)))  # 增广矩阵
    n = len(A)
    n1 = A1.shape[1]

    # 系数矩阵化为上三角
    for i in range(n - 1):
        # 找到当前列的主元
        s = np.argmax(np.abs(A1[i:n, i])) + i
        # 交换行
        A1[[i, s], :] = A1[[s, i], :]

        # 进行消元操作
        for k in range(i + 1, n):
            lik = A1[k, i] / A1[i, i]  # 算子
            A1[k, i:n1] -= lik * A1[i, i:n1]

    # 得到上三角后进行回代
    Ae = A1[:, :n]
    be = A1[:, n1 - 1]
    X = reg_utm(Ae, be)

    return X, Ae, be


# 测试具体矩阵
A2 = np.array([[10, 1, 2, 3, 4],
                [1, 9, -1, 2, -3],
                [2, -1, 7, 3, -5],
                [3, 2, 3, 12, -1],
                [4, -3, -5, -1, 15]], dtype=float)

b = np.array([12, -27, 14, -17, 12], dtype=float)

# 使用 gsem_column 函数求解
x2, Ae, be = gsem_column(A2, b)

# 输出结果
print("解向量 x2:", x2)
print("上三角矩阵 Ae:\n", Ae)
print("对应的右端项 be:", be)

解向量 x2: [ 1. -2.  3. -2.  1.]
上三角矩阵 Ae:
 [[10.          1.          2.          3.          4.        ]
 [ 0.          8.9        -1.2         1.7        -3.4       ]
 [ 0.          0.          6.43820225  2.62921348 -6.25842697]
 [ 0.          0.          0.          9.70157068  1.0052356 ]
 [ 0.          0.          0.          0.          5.91329376]]
对应的右端项 be: [ 12.         -28.2          7.79775281 -18.39790576   5.91329376]


In [6]:
#全主元消去法
def max_loc(A):
    """
    获取矩阵中最大元素的位置
    返回最大元素的值和其在矩阵中的位置
    """
    r, l = A.shape
    if r == 1:  # 行向量
        a = 1
        m = np.max(A)
        b = np.argmax(A)
    elif l == 1:  # 列向量
        b = 1
        m = np.max(A)
        a = np.argmax(A)
    else:
        M = np.max(A, axis=0)
        m = np.max(M)
        b = np.argmax(M)
        a = np.argmax(A[:, b])

    return m, a, b


def gsem_complete(A, b):
    """
    全主元消去法
    A : 系数矩阵
    b : 右端常数 [列向量]
    X : 求得的解向量
    Ae: 得到的上三角矩阵
    be: 对应的右端项
    """
    A1 = np.hstack((A, b.reshape(-1, 1)))  # 增广矩阵
    n = len(A)
    n1 = A1.shape[1]
    X = np.zeros(n)
    t = np.arange(n)  # 用于修正解的位置与原方程不匹配

    # 系数矩阵化为上三角的过程
    for i in range(n - 1):
        A_sub = A1[i:n, i:n]
        m, a, b = max_loc(A_sub)  # 获取最大元素位置
        a += i  # 修正为全矩阵的行索引
        b += i  # 修正为全矩阵的列索引
        A1[[i, a], :] = A1[[a, i], :]  # 交换行
        A1[:, [i, b]] = A1[:, [b, i]]  # 交换列
        t[i], t[b] = t[b], t[i]  # 修正解的位置

        # 进行消元操作
        for k in range(i + 1, n):
            lik = A1[k, i] / A1[i, i]  # 算子
            A1[k, i:n1] -= lik * A1[i, i:n1]

    # 得到上三角后进行回代
    Ae = A1[:, :n]
    be = A1[:, n1 - 1]
    X[t] = reg_utm(Ae, be)  # 使解匹配原方程位置

    return X, Ae, be


# 测试具体矩阵
A = np.array([[1, 2, 3],
              [2, 3, 1],
              [3, 1, 2]], dtype=float)

b = np.array([9, 8, 7], dtype=float)

# 使用 gsem_complete 函数求解
X, Ae, be = gsem_complete(A, b)

# 输出结果
print("解向量 X:", X)
print("上三角矩阵 Ae:\n", Ae)
print("对应的右端项 be:", be)

# 验证结果
result = np.dot(A, X)
print("验证：A @ X =", result)
print("应等于 b:", b)


解向量 X: [0.66666667 1.66666667 1.66666667]
上三角矩阵 Ae:
 [[ 3.          1.          2.        ]
 [ 0.          2.33333333 -0.33333333]
 [ 0.          0.          2.57142857]]
对应的右端项 be: [7.         3.33333333 4.28571429]
验证：A @ X = [9. 8. 7.]
应等于 b: [9. 8. 7.]
